0. Scan glass slides at 20x
1. Create Excel sheet with patient information
2. Screen H&E images to select potentially healthy sections
3. Annotate the Region of Interest (ROI) (Optional)
4. Convert ROI XML to Image (Code: roixml2png.py)
5. Convert H&E images to 1um resolution tif to apply DeepLab (Code: rescale_wsi.py)
6. Annotate 12 tissue compartments, apply DeepLab, and evaluate performance metrics
7. Apply HoVerNet
8. Rotate and crop tissue pieces and DLmask (Code)
9. Rotate HoVerNet segmented nuclei mask (Code)
10. Calculate tissue composition (Code)
11. Calculate epidermis thickness, waviness, roughness
12. Calculate dermis thickness, waviness
13. Calculate ECM anisotropy,Interstitial orientation, Thick gap ratio, Thick gap average
14. Calculate tissue morphology
15. ECM cell count and morphology + std using Manual Threshold method
16. Distance from ECM cell to tissue
17. All celltype morphology, orientation, and distance to tissue using HoVerNet
18. Further add features such as more detailed cell subtyping

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from natsort import natsorted

#utilities
from roixml2png import roixml2png
from rescale_wsi import rescale_wsi
from align_he_dl_cnt import align_he_dl_cnt

In [2]:
NDPI = r'\\fatherserverdw\kyuex\clue images'
oneum = r'\\fatherserverdw\kyuex\clue images\1um'
imcrop = r'\\fatherserverdw\kyuex\clue images\1um\imcrop'
imcroproi = r'\\fatherserverdw\kyuex\clue images\1um\imcrop_roi'

dlmask = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dlcrop = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop'
dlcroproi = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined\dlcrop_roi'

roixml = r'\\fatherserverdw\kyuex\clue images\annotations\roi'
roimask_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
tissue_area_small = r'\\fatherserverdw\kyuex\clue images\annotations\roi\TA_20rsf'

twelve_annot = r'\\fatherserverdw\kyuex\clue images\annotations\12class'

LUT = r"\\fatherserverdw\kyuex\imlist_all.xlsx"

In [4]:
#Load healthy tissue look up table (LUT)
LUT = pd.read_excel(LUT)
# xmlist = LUT['filename'][(LUT['student score']>1) & (LUT['Block or Slide?']=="Both")]
LUThealthy = LUT[(LUT['student score']>1)]

In [5]:
#Step 4: ROI XML 2 Image
#Saves Image to where XML is in subfolders
LUThealthyback = LUThealthy[LUThealthy['body part 1'].str.lower()=="back"]
xmlist = [os.path.join(roixml,xmlpth.replace('ndpi','xml')) for xmlpth in LUThealthyback['filename']]
[roixml2png(xmlpth,NDPI) for xmlpth in xmlist]
# pd.DataFrame(np.array(roiinfo),columns=['fn','ROIA','TA','ratio']).to_csv(r"\\fatherserverdw\kyuex\ROITA_ratio.csv")

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
#Step 5: rescale wsi to 1um for applying deeplab
#Saves Image to where wsi is in subfolder
wsis = [os.path.join(NDPI,_) for _ in LUThealthy.filename]
wsis = natsorted(wsis)
for idx,wsi in enumerate(wsis):
    rescale_wsi(wsi,1)
    print(idx,'/',len(wsis))

0 / 216
1 / 216
2 / 216
3 / 216
4 / 216
5 / 216
6 / 216
7 / 216
8 / 216
9 / 216
10 / 216
11 / 216
12 / 216
13 / 216
14 / 216
15 / 216
16 / 216
17 / 216
18 / 216
19 / 216
20 / 216
21 / 216
22 / 216
23 / 216
24 / 216
25 / 216
26 / 216
27 / 216
28 / 216
29 / 216
30 / 216
31 / 216
32 / 216
33 / 216
34 / 216
35 / 216
36 / 216
37 / 216
38 / 216
39 / 216
40 / 216
41 / 216
42 / 216
43 / 216
44 / 216
45 / 216
46 / 216
47 / 216
48 / 216
49 / 216
50 / 216
51 / 216
52 / 216
53 / 216
54 / 216
55 / 216
56 / 216
57 / 216
58 / 216
59 / 216
60 / 216
61 / 216
62 / 216
63 / 216
64 / 216
65 / 216
66 / 216
67 / 216
68 / 216
69 / 216
70 / 216
71 / 216
72 / 216
73 / 216
74 / 216
75 / 216
76 / 216
77 / 216
78 / 216
79 / 216
80 / 216
81 / 216
82 / 216
83 / 216
84 / 216
85 / 216
86 / 216
87 / 216
88 / 216
89 / 216
90 / 216
91 / 216
92 / 216
93 / 216
94 / 216
95 / 216
96 / 216
97 / 216
98 / 216
99 / 216
100 / 216
101 / 216
102 / 216
103 / 216
104 / 216
105 / 216
106 / 216
107 / 216
108 / 216
109 / 216
110 / 216


In [3]:
#8. Rotate and crop tissue pieces and DLmask (Code)
#9. Rotate HoVerNet segmented nuclei mask (Code)

LUTpth = r"\\fatherserverdw\kyuex\datadst\WSI_rotation_crop_LUT.xlsx"
ClUedegrot_df = pd.read_excel(LUTpth).astype("object")

In [4]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
def hasjson(row):
    fn,ext = os.path.splitext(row)
    return os.path.exists(os.path.join(cntsrc,fn+'.json'))
ClUedegrot_df['hasjson']=ClUedegrot_df['filename'].apply(lambda row: hasjson(row))

In [9]:
ClUedegrot_df2 = ClUedegrot_df[ClUedegrot_df['hasjson']==True]
ClUedegrot_df2 = ClUedegrot_df2[0:1]

In [10]:
len(ClUedegrot_df2)

72

In [6]:
cntsrc = r'\\fatherserverdw\kyuex\clue images\hovernet_out\json'
roisrc = r'\\fatherserverdw\kyuex\clue images\annotations\roi\labeledmask_20rsf'
wsisrc = r'\\fatherserverdw\kyuex\clue images'
dlsrc = r'\\fatherserverdw\kyuex\clue images\1um\classification_v9_combined'
dst = r'\\fatherserverdw\kyuex\datadst\20220929'

for idx,row in ClUedegrot_df2.iterrows():
    fn,ext = os.path.splitext(row['filename'])
    rotationdf = align_he_dl_cnt(dst,fn,wsisrc,dlsrc,cntsrc,roisrc)

section N:  1 / 2
section N:  2 / 2


In [7]:
rotationdf

,fn,secN,drot,bbox,bbox2
0,2022-06-09 22.06.26,1,86.695717,"[7728, 17087, 32800, 37615]","[216, 4933, 227, 9504]"
1,2022-06-09 22.06.26,2,78.047899,"[8128, 18271, 12528, 18847]","[1791, 7641, 833, 10187]"
